# Orchestrator Agent — End-to-end tests
This notebook exercises the Orchestrator to plan and run multi-step flows with checkpoints (human-in-the-loop).

In [3]:
# Setup imports and path
import sys, os, json
sys.path.append('..')
from importlib import reload
from agents.orchestra import Orchestrator

In [4]:
# Initialize orchestrator and register project defaults
orch = Orchestrator()
orch.register_defaults()
print('Registered actions:', sorted(list(orch.tools.keys())))

2025-08-24 18:11:33 | INFO | agent | Initializing Gmail service with credentials: /home/timmy/ai-teacher-assistant/credentials.json
2025-08-24 18:11:33 | INFO | agent | Gmail service initialized successfully
2025-08-24 18:11:33 | INFO | agent | Gmail service initialized successfully
2025-08-24 18:11:34 | INFO | agent | Gmail service initialized successfully. Sender: timmythaw17@gmail.com
2025-08-24 18:11:34 | INFO | agent | Gmail service initialized successfully. Sender: timmythaw17@gmail.com


Registered actions: ['create_google_form', 'draft_email', 'generate_assessment', 'generate_lesson_plan', 'render_assessment_markdown', 'render_lesson_markdown', 'schedule_calendar', 'send_email', 'suggest_timetable']


## Lesson flow with checkpoints
We will generate a lesson plan, render markdown, and suggest a timetable.
The run will pause after generating the plan and again before scheduling to calendar.

In [5]:
# Build options for lesson + timetable
outline_path = os.environ.get('COURSE_OUTLINE_PATH', '/home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf')
opts = {
    'lesson_input': {
        'sources': { 'course_outline': outline_path },
        'duration_weeks': 8,
        'class_size': 30,
        'sections_per_week': 1,
    },
    'timetable_opts': {
        'slot_hours': 1,
        'work_hours': [9, 17],
        'calendar_id': 'primary',
        'location_hint': 'Room 101',
    }
}

plan = orch.plan('Create a lesson plan and timetable', options=opts)
print('Planned tasks:', [ (t['id'], t['action']) for t in plan['tasks'] ])

state = orch.run(plan)
print('Run state:', state['state'])
if state['state']['status'] == 'paused':
    print('Paused after task:', state['state'].get('wait_for'))
    # Resume from pause
    state2 = orch.run(state)
    print('Run state (2):', state2['state'])
    if state2['state']['status'] == 'paused':
        print('Paused after task:', state2['state'].get('wait_for'))
        # At this point we have suggested timetable. Preview first slots and stop before scheduling
        tts = [t for t in state2['tasks'] if t['action'] == 'suggest_timetable']
        if tts and isinstance(tts[0].get('result'), dict):
            slots = (tts[0]['result'].get('suggested_slots') or [])[:5]
            print('Suggested slots (first 5):')
            for s in slots:
                print('-', s)

2025-08-24 18:12:13 | INFO | agent | LessonPlanAgent started with inputs: {'course_outline': '/home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf'}
2025-08-24 18:12:13 | INFO | agent | Extracted course outline: /home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf (length=3397)
2025-08-24 18:12:13 | INFO | agent | Extracted course outline: /home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf (length=3397)


Planned tasks: [('t1', 'generate_lesson_plan'), ('t2', 'render_lesson_markdown'), ('t3', 'suggest_timetable'), ('t4', 'schedule_calendar')]


2025-08-24 18:12:29 | INFO | agent | LessonPlanAgent successfully generated lesson plan (weeks=8, sections_per_week=1)


Run state: {'status': 'paused', 'wait_for': 't1'}
Paused after task: t1


2025-08-24 18:12:30 | INFO | agent | Detected calendar timezone: Asia/Bangkok
2025-08-24 18:12:30 | INFO | agent | Fetching calendar events for next 56 days
2025-08-24 18:12:30 | INFO | agent | Fetching calendar events for next 56 days
2025-08-24 18:12:31 | INFO | agent | Retrieved 50 events from Google Calendar
2025-08-24 18:12:31 | INFO | agent | Suggested 8 slots over 8 week(s)
2025-08-24 18:12:31 | INFO | agent | Retrieved 50 events from Google Calendar
2025-08-24 18:12:31 | INFO | agent | Suggested 8 slots over 8 week(s)


Run state (2): {'status': 'paused', 'wait_for': 't3'}
Paused after task: t3
Suggested slots (first 5):
- {'start': '2025-08-25T16:00', 'end': '2025-08-25T17:00', 'title': 'Week 1', 'reason': 'Consistent weekly slot on Mon 16:00 avoiding conflicts', 'location': 'Room 101'}
- {'start': '2025-09-01T16:00', 'end': '2025-09-01T17:00', 'title': 'Week 2', 'reason': 'Consistent weekly slot on Mon 16:00 avoiding conflicts', 'location': 'Room 101'}
- {'start': '2025-09-08T16:00', 'end': '2025-09-08T17:00', 'title': 'Week 3', 'reason': 'Consistent weekly slot on Mon 16:00 avoiding conflicts', 'location': 'Room 101'}
- {'start': '2025-09-15T16:00', 'end': '2025-09-15T17:00', 'title': 'Week 4', 'reason': 'Consistent weekly slot on Mon 16:00 avoiding conflicts', 'location': 'Room 101'}
- {'start': '2025-09-22T16:00', 'end': '2025-09-22T17:00', 'title': 'Week 5', 'reason': 'Consistent weekly slot on Mon 16:00 avoiding conflicts', 'location': 'Room 101'}


## Assessment flow with checkpoint
We will generate an assessment and render markdown. The run pauses after generating the assessment (before creating a Google Form).

In [6]:
# Prepare assessment options
lecture_pdf = '/home/timmy/ai-teacher-assistant/static/Lecture_Slide.pdf'
opts2 = {
    'assessment_input': {
        'source': lecture_pdf,
        'spec': { 'type': 'MCQ', 'difficulty': 'Medium', 'count': 5, 'rubric': True }
    }
}

plan2 = orch.plan('Generate an assessment from a lecture PDF', options=opts2)
print('Planned tasks:', [ (t['id'], t['action']) for t in plan2['tasks'] ])

stateA = orch.run(plan2)
print('Run state:', stateA['state'])
if stateA['state']['status'] == 'paused':
    # The assessment should be available now; show summary
    at = [t for t in stateA['tasks'] if t['action'] == 'generate_assessment']
    if at and isinstance(at[0].get('result'), dict):
        qn = len(at[0]['result'].get('questions') or [])
        print('Assessment questions:', qn)
    md = [t for t in stateA['tasks'] if t['action'] == 'render_assessment_markdown']
    if md and isinstance(md[0].get('result'), str):
        print('Markdown length:', len(md[0]['result']))
    # Stop here to avoid creating a Google Form in tests

2025-08-24 18:12:51 | INFO | agent | LessonPlanAgent started with inputs: {'type': 'MCQ', 'difficulty': 'Medium', 'count': 5, 'rubric': True}
2025-08-24 18:12:51 | INFO | agent | Extracted text from PDF: /home/timmy/ai-teacher-assistant/static/Lecture_Slide.pdf (length=7438)
2025-08-24 18:12:51 | INFO | agent | Extracted text from PDF: /home/timmy/ai-teacher-assistant/static/Lecture_Slide.pdf (length=7438)


Planned tasks: [('t1', 'generate_assessment'), ('t2', 'render_assessment_markdown'), ('t3', 'create_google_form')]


2025-08-24 18:12:56 | INFO | agent | AssessmentAgent successfully generated assessment with 5 questions


{
  "title": "Assessment on Software Requirements and SDLC Models",
  "type": "MCQ",
  "difficulty": "Medium",
  "questions": [
    {
      "q": "Which of the following best describes the Software Development Life Cycle (SDLC)?",
      "options": [
        "A programming language used to develop software",
        "A framework defining tasks performed at each step in the software development process",
        "A testing methodology for debugging software",
        "A document that contains only user requirements"
      ],
      "answer": "A framework defining tasks performed at each step in the software development process"
    },
    {
      "q": "In the Waterfall Model, which statement is true?",
      "options": [
        "Phases can overlap and run in parallel",
        "The next phase starts only after the previous phase has finished",
        "It emphasizes iterative development",
        "It is primarily used in Agile methodologies"
      ],
      "answer": "The next phase start

## Prompt-driven runs
Use plain-language prompts to plan and run tasks. These examples use static files for inputs to keep tests reproducible.

In [7]:
# Helpers: ensure orchestrator is available, pretty-print tasks, and run a prompt
try:
    orch
except NameError:
    import sys, os
    sys.path.append('..')
    from agents.orchestra import Orchestrator
    orch = Orchestrator()
    orch.register_defaults()
    print("Orchestrator initialized and defaults registered.")

def pp_tasks(state_or_plan):
    tasks = state_or_plan.get('tasks', [])
    for t in tasks:
        tid = t.get('id')
        action = t.get('action')
        status = t.get('status', 'planned')
        print(f"{tid} | {action} | status={status}")

def run_prompt(prompt: str, options: dict | None = None, resume: bool = False):
    print("Prompt:", prompt)
    plan = orch.plan(prompt, options=options or {})
    print("Planned tasks:")
    pp_tasks(plan)
    state = orch.run(plan)
    print("Run state:", state.get('state'))
    if resume and state.get('state', {}).get('status') == 'paused':
        state2 = orch.run(state)
        print("Run state (2):", state2.get('state'))
        return state2
    return state

### Example 1: Lesson plan and timetable (pauses before scheduling)
Uses the course outline PDF in `static/` and suggests a timetable. The run pauses prior to adding events to Google Calendar.

In [8]:
lesson_prompt = "Create a lesson plan and timetable"
outline_path = os.environ.get('COURSE_OUTLINE_PATH', '/home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf')
lesson_options = {
    'lesson_input': {
        'sources': { 'course_outline': outline_path },
        'duration_weeks': 8,
        'class_size': 30,
        'sections_per_week': 1,
    },
    'timetable_opts': {
        'slot_hours': 1,
        'work_hours': [9, 17],
        'calendar_id': 'primary',
        'location_hint': 'Room 101',
    }
}

state_lesson = run_prompt(lesson_prompt, lesson_options, resume=False)
if state_lesson.get('state', {}).get('status') == 'paused':
    # Preview suggested slots if available
    tts = [t for t in state_lesson.get('tasks', []) if t.get('action') == 'suggest_timetable']
    if tts and isinstance(tts[0].get('result'), dict):
        slots = (tts[0]['result'].get('suggested_slots') or [])[:5]
        print('Suggested slots (first 5):')
        for s in slots:
            print('-', s)
    else:
        print('No suggested slots found yet.')

2025-08-24 18:16:59 | INFO | agent | LessonPlanAgent started with inputs: {'course_outline': '/home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf'}
2025-08-24 18:16:59 | INFO | agent | Extracted course outline: /home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf (length=3397)
2025-08-24 18:16:59 | INFO | agent | Extracted course outline: /home/timmy/ai-teacher-assistant/static/SRAS_Course_Outline.pdf (length=3397)


Prompt: Create a lesson plan and timetable
Planned tasks:
t1 | generate_lesson_plan | status=pending
t2 | render_lesson_markdown | status=pending
t3 | suggest_timetable | status=pending
t4 | schedule_calendar | status=pending


2025-08-24 18:17:13 | INFO | agent | LessonPlanAgent successfully generated lesson plan (weeks=8, sections_per_week=1)


Run state: {'status': 'paused', 'wait_for': 't1'}
No suggested slots found yet.


### Example 2: Assessment generation (pauses before creating Google Form)
Uses a static lecture PDF to generate an assessment and render markdown, then pauses before creating a Google Form.

In [9]:
assessment_prompt = "Generate an assessment from a lecture PDF and prepare a Google Form"
lecture_pdf = '/home/timmy/ai-teacher-assistant/static/Lecture_Slide.pdf'
assessment_options = {
    'assessment_input': {
        'source': lecture_pdf,
        'spec': { 'type': 'MCQ', 'difficulty': 'Medium', 'count': 5, 'rubric': True }
    }
}

state_assessment = run_prompt(assessment_prompt, assessment_options, resume=False)
if state_assessment.get('state', {}).get('status') == 'paused':
    at = [t for t in state_assessment.get('tasks', []) if t.get('action') == 'generate_assessment']
    if at and isinstance(at[0].get('result'), dict):
        qn = len(at[0]['result'].get('questions') or [])
        print('Assessment questions:', qn)
    md = [t for t in state_assessment.get('tasks', []) if t.get('action') == 'render_assessment_markdown']
    if md and isinstance(md[0].get('result'), str):
        print('Markdown length:', len(md[0]['result']))
    else:
        print('No markdown result yet.')

2025-08-24 18:17:38 | INFO | agent | LessonPlanAgent started with inputs: {'type': 'MCQ', 'difficulty': 'Medium', 'count': 5, 'rubric': True}
2025-08-24 18:17:38 | INFO | agent | Extracted text from PDF: /home/timmy/ai-teacher-assistant/static/Lecture_Slide.pdf (length=7438)
2025-08-24 18:17:38 | INFO | agent | Extracted text from PDF: /home/timmy/ai-teacher-assistant/static/Lecture_Slide.pdf (length=7438)


Prompt: Generate an assessment from a lecture PDF and prepare a Google Form
Planned tasks:
t1 | generate_assessment | status=pending
t2 | render_assessment_markdown | status=pending
t3 | create_google_form | status=pending


2025-08-24 18:17:43 | INFO | agent | AssessmentAgent successfully generated assessment with 5 questions


{
  "title": "Assessment on Software Development Life Cycle and Requirements Engineering",
  "type": "MCQ",
  "difficulty": "Medium",
  "questions": [
    {
      "q": "Which of the following best describes the Software Development Life Cycle (SDLC)?",
      "options": [
        "A programming language used for software development",
        "A framework defining tasks performed at each step in the software development process",
        "A tool for debugging software",
        "A testing methodology for software products"
      ],
      "answer": "A framework defining tasks performed at each step in the software development process"
    },
    {
      "q": "In the Waterfall Model, what is the purpose of the Requirement Specification phase?",
      "options": [
        "To design the system architecture",
        "To gather and document customer requirements in the SRS document",
        "To test the integrated system",
        "To release patches for maintenance"
      ],
      "answer

# Orchestrator Agent Gradio Chat Test
This block launches a Gradio chat interface to interactively test the Orchestrator agent. Type a prompt (e.g., "Create a lesson plan and timetable" or "Generate an assessment from a lecture PDF") and optionally upload a PDF. The chat will show Markdown previews and pause at checkpoints.

In [3]:
import gradio as gr
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from agents.orchestra import Orchestrator

orch = Orchestrator()
orch.register_defaults()

def chat_orchestrator(user_msg, file=None, state=None):
    options = {}
    if file is not None:
        up_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'uploads'))
        os.makedirs(up_dir, exist_ok=True)
        fname = os.path.basename(file.name)
        dest = os.path.join(up_dir, fname)
        with open(dest, 'wb') as f:
            f.write(file.read())
        if 'lesson' in user_msg.lower():
            options['lesson_input'] = {'sources': {'course_outline': dest}}
        elif 'assessment' in user_msg.lower():
            options['assessment_input'] = {'source': dest, 'spec': {'type': 'MCQ', 'difficulty': 'Medium', 'count': 5, 'rubric': True}}
    plan = orch.plan(user_msg, options=options)
    state = orch.run(plan)
    if state['state']['status'] == 'paused':
        msg = f"Paused after: {state['state'].get('wait_for')}\n"
        for t in state['tasks']:
            if t['action'].startswith('render_') and isinstance(t.get('result'), str):
                msg += t['result']
        return msg, state
    msg = ''
    for t in state['tasks']:
        if t['action'].startswith('render_') and isinstance(t.get('result'), str):
            msg += t['result']
    if not msg:
        msg = str(state)
    return msg, state

def gradio_chat():
    with gr.Blocks() as demo:
        gr.Markdown("# Orchestrator Agent Chat Demo\nType a prompt and optionally upload a PDF.")
        chat = gr.ChatInterface(
            fn=chat_orchestrator,
            additional_inputs=[gr.File(label="Attach PDF (optional)")],
            examples=[
                ["Create a lesson plan and timetable"],
                ["Generate an assessment from a lecture PDF"],
            ],
        )
    demo.launch()

gradio_chat()

2025-09-04 00:50:12 | INFO | agent | Initializing Gmail service with credentials: /home/timmy/ai-teacher-assistant/credentials.json
2025-09-04 00:50:12 | WARNING | agent | Failed to refresh credentials: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})
2025-09-04 00:50:12 | INFO | agent | Starting OAuth flow for Gmail authentication
2025-09-04 00:50:12 | WARNING | agent | Failed to refresh credentials: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})
2025-09-04 00:50:12 | INFO | agent | Starting OAuth flow for Gmail authentication
gio: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=840970484260-ja0ddn2vaolpbbmb87pteahiuufs8urr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A33105%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googlea

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=840970484260-ja0ddn2vaolpbbmb87pteahiuufs8urr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A33105%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=jZu3Af8PKs2gHoPTWTZM9UsZCOMkDK&access_type=offline


2025-09-04 00:50:22 | INFO | agent | Gmail OAuth flow completed successfully
2025-09-04 00:50:22 | INFO | agent | Gmail credentials cached at /home/timmy/ai-teacher-assistant/token.json
2025-09-04 00:50:22 | INFO | agent | Gmail credentials cached at /home/timmy/ai-teacher-assistant/token.json
2025-09-04 00:50:22 | INFO | agent | Gmail service initialized successfully
2025-09-04 00:50:22 | INFO | agent | Gmail service initialized successfully
2025-09-04 00:50:23 | INFO | agent | Gmail service initialized successfully. Sender: timmythaw17@gmail.com
2025-09-04 00:50:23 | INFO | agent | Gmail service initialized successfully. Sender: timmythaw17@gmail.com
/home/timmy/ai-teacher-assistant/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
/home/tim

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/timmy/ai-teacher-assistant/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 667, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/timmy/ai-teacher-assistant/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/timmy/ai-teacher-assistant/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/timmy/ai-teacher-assistant/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1779, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/timmy/ai-teacher-assistant/.venv/lib/python3.12/site-packages/gradio/utils.py", line 8